# Language translation : seq2seq

[參考連結](https://github.com/deep-diver/EN-FR-MLT-tensorflow/blob/master/dlnd_language_translationv2.ipynb)

## Preprocessing

In [1]:
import os
import pickle
import copy
import numpy as np

def load_data(path):
    input_file = os.path.join(path)
    with open(input_file, 'r', encoding='utf-8') as f:
        data = f.read()

    return data

In [42]:
source_path = '../data/giga-fren/small_vocab_en'
target_path = '../data/giga-fren/small_vocab_fr'
source_text = load_data(source_path)
target_text = load_data(target_path)

In [43]:
import numpy as np
from collections import Counter

print('Dataset Brief Stats')
print('* number of unique words in English sample sentences: {}\
        [this is roughly measured/without any preprocessing]'.format(len(Counter(source_text.split()))))
print()

english_sentences = source_text.split('\n')
print('* English sentences')
print('\t- number of sentences: {}'.format(len(english_sentences)))
print('\t- avg. number of words in a sentence: {}'.format(np.average([len(sentence.split()) for sentence in english_sentences])))

french_sentences = target_text.split('\n')
print('* French sentences')
print('\t- number of sentences: {} [data integrity check / should have the same number]'.format(len(french_sentences)))
print('\t- avg. number of words in a sentence: {}'.format(np.average([len(sentence.split()) for sentence in french_sentences])))
print()

sample_sentence_range = (0, 5)
side_by_side_sentences = list(zip(english_sentences, french_sentences))[sample_sentence_range[0]:sample_sentence_range[1]]
print('* Sample sentences range from {} to {}'.format(sample_sentence_range[0], sample_sentence_range[1]))

for index, sentence in enumerate(side_by_side_sentences):
    en_sent, fr_sent = sentence
    print('[{}-th] sentence'.format(index+1))
    print('\tEN: {}'.format(en_sent))
    print('\tFR: {}'.format(fr_sent))
    print()

Dataset Brief Stats
* number of unique words in English sample sentences: 227        [this is roughly measured/without any preprocessing]

* English sentences
	- number of sentences: 137861
	- avg. number of words in a sentence: 13.225277634719028
* French sentences
	- number of sentences: 137861 [data integrity check / should have the same number]
	- avg. number of words in a sentence: 14.226612312401622

* Sample sentences range from 0 to 5
[1-th] sentence
	EN: new jersey is sometimes quiet during autumn , and it is snowy in april .
	FR: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .

[2-th] sentence
	EN: the united states is usually chilly during july , and it is usually freezing in november .
	FR: les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .

[3-th] sentence
	EN: california is usually quiet during march , and it is usually hot in june .
	FR: california est généralement calme en mars , et il est généraleme

In [44]:
def  text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = [[source_vocab_to_int[w] for w in line.split()] for line in source_text.split('\n')]
    
    id_eos = target_vocab_to_int['<EOS>']
    
    target_id_text = [[target_vocab_to_int[w] for w in line.split()] + [id_eos] for line in target_text.split('\n')]
    
    return source_id_text, target_id_text

In [45]:
CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }

def create_lookup_tables(text):
    # make a list of unique words
    vocab = set(text.split())

    # (1)
    # starts with the special tokens
    vocab_to_int = copy.copy(CODES)

    # the index (v_i) will starts from 4 (the 2nd arg in enumerate() specifies the starting index)
    # since vocab_to_int already contains special tokens
    for v_i, v in enumerate(vocab, len(CODES)):
        vocab_to_int[v] = v_i

    # (2)
    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab

In [46]:
def preprocess_and_save_data(source_path, target_path, text_to_ids):
    # Preprocess
    
    # load original data (English, French)
    source_text = load_data(source_path)
    target_text = load_data(target_path)

    # to the lower case
    source_text = source_text.lower()
    target_text = target_text.lower()

    # create lookup tables for English and French data
    source_vocab_to_int, source_int_to_vocab = create_lookup_tables(source_text)
    target_vocab_to_int, target_int_to_vocab = create_lookup_tables(target_text)

    # create list of sentences whose words are represented in index
    source_text, target_text = text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int)

    # Save data for later use
    pickle.dump((
        (source_text, target_text),
        (source_vocab_to_int, target_vocab_to_int),
        (source_int_to_vocab, target_int_to_vocab)), open('../data/giga-fren/preprocess.p', 'wb'))

## Save preprocess data

In [47]:
preprocess_and_save_data(source_path, target_path, text_to_ids)

## Load preprocess data

In [1]:
import pickle

def load_preprocess():
    with open('../data/giga-fren/preprocess.p', mode='rb') as in_file:
        return pickle.load(in_file)

import numpy as np

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()

---

## Seq2Seq with attention - Keras model

- https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043
- https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/
- http://adventuresinmachinelearning.com/word2vec-keras-tutorial/
- https://medium.com/@dev.elect.iitd/neural-machine-translation-using-word-level-seq2seq-model-47538cba8cd7

In [2]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

def _time_distributed_dense(x, w, b=None, dropout=None,
                            input_dim=None, output_dim=None,
                            timesteps=None, training=None):
    """Apply `y . w + b` for every temporal slice y of x.
    # Arguments
        x: input tensor.
        w: weight matrix.
        b: optional bias vector.
        dropout: wether to apply dropout (same dropout mask
            for every temporal slice of the input).
        input_dim: integer; optional dimensionality of the input.
        output_dim: integer; optional dimensionality of the output.
        timesteps: integer; optional number of timesteps.
        training: training phase tensor or boolean.
    # Returns
        Output tensor.
    """
    if not input_dim:
        input_dim = K.shape(x)[2]
    if not timesteps:
        timesteps = K.shape(x)[1]
    if not output_dim:
        output_dim = K.shape(w)[1]

    if dropout is not None and 0. < dropout < 1.:
        # apply the same dropout pattern at every timestep
        ones = K.ones_like(K.reshape(x[:, 0, :], (-1, input_dim)))
        dropout_matrix = K.dropout(ones, dropout)
        expanded_dropout_matrix = K.repeat(dropout_matrix, timesteps)
        x = K.in_train_phase(x * expanded_dropout_matrix, x, training=training)

    # collapse time dimension and batch dimension together
    x = K.reshape(x, (-1, input_dim))
    x = K.dot(x, w)
    if b is not None:
        x = K.bias_add(x, b)
    # reshape to 3D tensor
    if K.backend() == 'tensorflow':
        x = K.reshape(x, K.stack([-1, timesteps, output_dim]))
        x.set_shape([None, None, output_dim])
    else:
        x = K.reshape(x, (-1, timesteps, output_dim))
    return x

class AttentionDecoder(Recurrent):
 
    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space
 
        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
 
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
 
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
 
        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences
 
    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """
 
        self.batch_size, self.timesteps, self.input_dim = input_shape
 
        if self.stateful:
            super(AttentionDecoder, self).reset_states()
 
        self.states = [None, None]  # y, s
 
        """
            Matrices for creating the context vector
        """
 
        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
 
        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
 
        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
 
        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True
 
    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x
 
        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = _time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)
 
        return super(AttentionDecoder, self).call(x)
 
    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))
 
        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])
 
        return [y0, s0]
 
    def step(self, x, states):
 
        ytm, stm = states
 
        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)
 
        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)
 
        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)
 
        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:
 
        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)
 
        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)
 
        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)
 
        # new hidden state:
        st = (1-zt)*stm + zt * s_tp
 
        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)
 
        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]
 
    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)
 
    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.utils import to_categorical
from keras.layers import LSTM
from random import randint
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from sklearn.model_selection import train_test_split

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

def padding(seq,padding_length,padding_int):
    zero_ar = np.ones((len(seq),padding_length))*padding_int
    for i,s in enumerate(seq):
        for j,k in enumerate(s): #s <= padding_length
            zero_ar[i,j] = k
    return zero_ar

# configure problem
n_features = 50
n_timesteps_in = 5
n_timesteps_out = 2
n_epochs = 20
source_vocab_size = len(source_vocab_to_int)
target_vocab_size = len(target_vocab_to_int)
max_source_len = max([len(s) for s in source_int_text])
max_target_len = max([len(s) for s in target_int_text])
padding_size = max(max_source_len,max_target_len)
max_dim = max(source_vocab_size,target_vocab_size)


X = to_categorical(padding(source_int_text,padding_size,source_vocab_to_int['<PAD>']))

y = to_categorical(padding(target_int_text,padding_size,target_vocab_to_int['<PAD>']))

train_x,valid_x,train_y,valid_y = train_test_split(X,y,test_size = 0.2, shuffle = True, random_state = 42)


In [7]:

# define model
model = Sequential()
model.add(LSTM(128, input_shape=(padding_size, source_vocab_size),return_sequences = True))
model.add(LSTM(128,return_sequences = True))
model.add(AttentionDecoder(128, target_vocab_size))  #target_vocab_size
# model.add(LSTM(target_vocab_size,return_sequences = True))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

#128, 128  workable  -> 0.965 on validation set (not better than simple seq2seq model)
#64, 64  workable  -> 0.964 on validation set  
#128, 64, 64  -> 0.973 on validation set (huge)
#128,128, 128 -> 0.971 on validation set (huge2)
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 24, 128)           184320    
_________________________________________________________________
lstm_2 (LSTM)                (None, 24, 128)           131584    
_________________________________________________________________
AttentionDecoder (AttentionD (None, 24, 358)           505738    
Total params: 821,642
Trainable params: 821,642
Non-trainable params: 0
_________________________________________________________________


In [8]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(
    monitor='val_loss', 
    patience=3,verbose=0, 
    mode='auto')

model.fit(train_x, train_y, validation_split = 0.2, epochs=n_epochs,callbacks = [es], verbose=1)

Train on 88230 samples, validate on 22058 samples
Epoch 1/20
88230/88230 [==============================] - 358s 4ms/step - loss: 1.3400 - acc: 0.6449 - val_loss: 0.8405 - val_acc: 0.7501
Epoch 2/20
88230/88230 [==============================] - 355s 4ms/step - loss: 0.6695 - acc: 0.7985 - val_loss: 0.4809 - val_acc: 0.8530
Epoch 3/20
88230/88230 [==============================] - 355s 4ms/step - loss: 0.3698 - acc: 0.8885 - val_loss: 0.2883 - val_acc: 0.9136
Epoch 4/20
88230/88230 [==============================] - 355s 4ms/step - loss: 0.2223 - acc: 0.9344 - val_loss: 0.2258 - val_acc: 0.9330
Epoch 5/20
88230/88230 [==============================] - 355s 4ms/step - loss: 0.1711 - acc: 0.9493 - val_loss: 0.1944 - val_acc: 0.9422
Epoch 6/20
88230/88230 [==============================] - 355s 4ms/step - loss: 0.1437 - acc: 0.9576 - val_loss: 0.1370 - val_acc: 0.9601
Epoch 7/20
88230/88230 [==============================] - 347s 4ms/step - loss: 0.1324 - acc: 0.9614 - val_loss: 0.1295 - 

In [9]:
model.save('../saved_models/seq2seq_model_huge2.h5')

## Load model

In [5]:
from keras.models import load_model

model = load_model('../saved_models/seq2seq_model_huge.h5',custom_objects={'AttentionDecoder':AttentionDecoder})

### Overall Accuracy on validation set

In [10]:
# test on validation set

# takes few seconds to calculate acc.
valid_acc = np.mean(np.equal(np.argmax(model.predict(valid_x),axis=2),np.argmax(valid_y,axis=2)))
print('Validation accuracy: %.3f'%valid_acc)

Validation accuracy: 0.971


### Translation on training set

In [11]:
target_int_to_vocab = {v:k for k,v in target_vocab_to_int.items()}
source_int_to_vocab = {v:k for k,v in source_vocab_to_int.items()}

# x and y_true should be the model input and output while fitting
# shape of x and y_true: (1,time_step,num_dim)
def model_predict(model,x, y_true, source_int_to_vocab,target_int_to_vocab):
    pred_y = np.argmax(model.predict(x),axis=2)
    print('Input: \r\n {}\r\n'.format(" ".join([source_int_to_vocab[i] for i in np.argmax(x[0],axis=1)])))
    print('Prdiction: \r\n{}\r\n'.format(" ".join([target_int_to_vocab[i] for i in pred_y[0]])))
    print('Answer: \r\n{}'.format(" ".join([target_int_to_vocab[i] for i in np.argmax(y_true[0],axis=1)])))
    
model_predict(model,train_x[0:1],train_y[0:1],source_int_to_vocab,target_int_to_vocab)

Input: 
 india is usually freezing during january , and it is never nice in autumn . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Prdiction: 
l' inde est le gel habituellement en janvier , et il est jamais agréable à l' automne . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD>

Answer: 
l' inde est le gel habituellement en janvier , et il est jamais agréable à l' automne . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD>


### Translation on validation set

In [12]:
model_predict(model,valid_x[0:1],valid_y[0:1],source_int_to_vocab,target_int_to_vocab)

Input: 
 california is sometimes dry during may , and it is sometimes wonderful in february . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Prdiction: 
californie est parfois sec au mois de mai , et il est parfois merveilleux en février . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Answer: 
californie est parfois sec au mois de mai , et il est parfois merveilleux en février . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
